In [13]:
import pandas as pd

In [14]:
df=pd.read_csv("../Integrated_data/merged_df.csv",encoding="utf-8")

In [15]:
# 널값을 가진 행 삭제
df = df.dropna(subset=['collect_datetime'])

## 결측치 처리

In [16]:
def interpolated(df):
    temp=[]
    for i in df['ID'].unique():
        temp.append(df[df['ID']==i].interpolate())
    new_df=pd.concat(temp,axis=0)
    return new_df

In [17]:
new_df=interpolated(df)

In [18]:
new_df.fillna(method='bfill',inplace=True)

# height 값 기준 이상치 탐지
1. 시계열 특성상 과거보다 키가 줄어들 수 없다.
2. 상식선에서 6개월 기간내에 155에서 180으로 약 16%로 이상 상승할 수 없으모 13프로 이상 상승한 값은 제외시킨다. 

In [19]:
def gap(first, last):
    gap_per=(last-first)/first*100
    return abs(gap_per)

In [20]:
def abmormaled(df):
    temp=[]
    for i in df['ID'].unique():
        temp_df=df[df['ID']==i].reset_index(drop=True)
        # 상승하지 않는 값은 직전 값으로 대체
        for i in range(1, len(temp_df)):
            if temp_df.at[i, 'height'] <= temp_df.at[i - 1, 'height']:
                temp_df.at[i, 'height'] = temp_df.at[i - 1, 'height']
            if gap(temp_df.at[i - 1, 'height'],temp_df.at[i, 'height'])>=16:
                temp_df.at[i, 'height'] = temp_df.at[i - 1, 'height']
        temp.append(temp_df)
    new_df2=pd.concat(temp,axis=0)
    return new_df2

In [21]:
new_df2=abmormaled(new_df)
new_df2.reset_index(drop=True,inplace=True)

In [22]:
new_df2

,ID,collect_datetime,gender,grade,height,weight,step_count,burned calory,eat_calory,Sleep_time
0,#AAGQKY,2022-07-14,2.0,4.0,148.0,45.15,72.0,8.877,2000.0,12.0
1,#AAGQKY,2022-07-15,2.0,4.0,148.0,45.15,72.0,8.877,2000.0,12.0
2,#AAGQKY,2022-07-16,2.0,4.0,148.0,45.15,72.0,8.877,2000.0,12.0
3,#AAGQKY,2022-07-17,2.0,4.0,148.0,45.15,72.0,8.877,2000.0,12.0
4,#AAGQKY,2022-07-18,2.0,4.0,148.0,45.15,72.0,8.877,2000.0,12.0
...,...,...,...,...,...,...,...,...,...,...
59509,#ZZLSSL,2023-01-07,1.0,6.0,153.0,68.00,NaN,NaN,NaN,NaN
59510,#ZZLSSL,2023-01-08,1.0,6.0,153.0,68.00,NaN,NaN,NaN,NaN
59511,#ZZLSSL,2023-01-09,1.0,6.0,153.0,68.00,NaN,NaN,NaN,NaN
59512,#ZZLSSL,2023-01-10,1.0,6.0,153.0,68.00,NaN,NaN,NaN,NaN


# 완전 결측치 전체 중앙값으로 대체
위에 있는 NULL값은 해당 ID에 값이 한개도 기록되지 않아 생기는 것 이므로 전체 중앙값으로 대체하여 성능에 영향을 주는 것을 최소화 시킨다.

In [23]:
#step_count 결측치 대체
new_df2["step_count"].fillna(new_df2["step_count"].median(),inplace=True)

#burned_calory 결측치 대체
new_df2["burned calory"].fillna(new_df2["burned calory"].median(),inplace=True)
#eat_calory 결측치 대체
new_df2["eat_calory"].fillna(new_df2["eat_calory"].median(),inplace=True)

#Sleep_time 결측치 대체
new_df2["Sleep_time"].fillna(new_df2["Sleep_time"].median(),inplace=True)

In [24]:
new_df2.to_csv("../Integrated_data/Final_dataset.csv",encoding="utf-8",index=False)